In [10]:
import datetime
import hopsworks
from helpers import util
import pandas as pd
import json
import warnings
warnings.filterwarnings("ignore")

In [11]:
project = hopsworks.login(engine="python")
fs = project.get_feature_store() 
secrets = hopsworks.get_secrets_api()

# This line will fail if you have not registered the AQICN_API_KEY as a secret in Hopsworks
AQICN_API_KEY = secrets.get_secret("AQICN_API_KEY").value
sensors_str = secrets.get_secret("SENSORS_JSON").value
sensors_data = json.loads(sensors_str)

today = datetime.date.today()

2025-11-10 15:14:54,205 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-11-10 15:14:54,210 INFO: Initializing external client
2025-11-10 15:14:54,212 INFO: Base URL: https://c.app.hopsworks.ai:443


2025-11-10 15:14:55,900 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1272014


In [12]:
# Retrieve feature groups
air_quality_fg = fs.get_feature_group(
    name='air_quality',
    version=1,
)
weather_fg = fs.get_feature_group(
    name='weather',
    version=1,
)

In [13]:
start_date = today - datetime.timedelta(days=4)

last_3_days_aq_query = (
    air_quality_fg.select(["city", "pm25", "date"]).filter((air_quality_fg.date >= str(start_date)) & (air_quality_fg.date <= str(today)))
)

aq_3_days_history_df = last_3_days_aq_query.read()

aq_dfs = []

for location in sensors_data:
    aqicn_url = location['aqicn_url']
    country = location['country']
    city = location['city']
    street = location['street']
    aq_location_today_df = util.get_pm25(aqicn_url, country, city, street, today, AQICN_API_KEY)
    for lagged_day in [1,2,3]:
        lag_date = today - pd.Timedelta(days=lagged_day)
        mask = (aq_3_days_history_df['city'] == city) \
            & (aq_3_days_history_df['date'].dt.date == lag_date)
        lag_value = aq_3_days_history_df.loc[mask, 'pm25']
        if not lag_value.empty:
            aq_location_today_df[f'lagged_{lagged_day}'] = lag_value.values[0]
        else:
            aq_location_today_df[f'lagged_{lagged_day}'] = None

    aq_dfs.append(aq_location_today_df)

aq_today_df = pd.concat(aq_dfs, ignore_index=True) 

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.54s) 


In [14]:
weather_dfs = []

for location in sensors_data:
    city = location['city']
    latitude = location['latitude']
    longitude = location['longitude']
    hourly_df = util.get_hourly_weather_forecast(city, latitude, longitude)
    hourly_df = hourly_df.set_index('date')

    # We will only make 1 daily prediction, so we will replace the hourly forecasts with a single daily forecast
    # We only want the daily weather data, so only get weather at 12:00
    daily_location_df = hourly_df.between_time('11:59', '12:01')
    daily_location_df = daily_location_df.reset_index()
    daily_location_df['date'] = pd.to_datetime(daily_location_df['date']).dt.date
    daily_location_df['date'] = pd.to_datetime(daily_location_df['date'])
    daily_location_df['city'] = city
    weather_dfs.append(daily_location_df)

daily_df = pd.concat(weather_dfs, ignore_index=True) 
daily_df.info()

Coordinates 59.5°N 13.5°E
Elevation 55.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s
Coordinates 59.75°N 13.75°E
Elevation 147.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s
Coordinates 59.5°N 12.25°E
Elevation 223.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s
Coordinates 59.5°N 14.25°E
Elevation 146.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28 entries, 0 to 27
Data columns (total 6 columns):
 #   Column                       Non-Null Count  Dtype         
---  ------                       --------------  -----         
 0   date                         28 non-null     datetime64[ns]
 1   temperature_2m_mean          28 non-null     float32       
 2   precipitation_sum            28 non-null     float32       
 3   wind_speed_10m_max           28 non-null     float32       
 4   wind_direction_10m_dominant  28 non-null     float32       
 5   city                         28 non

In [15]:
# Insert new data
air_quality_fg.insert(aq_today_df)
weather_fg.insert(daily_df, wait=True)

2025-11-10 15:15:13,284 INFO: 	1 expectation(s) included in expectation_suite.
Validation succeeded.
Validation Report saved successfully, explore a summary at https://c.app.hopsworks.ai:443/p/1272014/fs/1258613/fg/1668629


Uploading Dataframe: 100.00% |██████████| Rows 4/4 | Elapsed Time: 00:01 | Remaining Time: 00:00


Launching job: air_quality_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1272014/jobs/named/air_quality_1_offline_fg_materialization/executions
2025-11-10 15:15:27,377 INFO: 	2 expectation(s) included in expectation_suite.
Validation succeeded.
Validation Report saved successfully, explore a summary at https://c.app.hopsworks.ai:443/p/1272014/fs/1258613/fg/1668630


Uploading Dataframe: 100.00% |██████████| Rows 28/28 | Elapsed Time: 00:01 | Remaining Time: 00:00


Launching job: weather_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1272014/jobs/named/weather_1_offline_fg_materialization/executions
2025-11-10 15:16:10,689 INFO: Waiting for execution to finish. Current state: SUBMITTED. Final status: UNDEFINED
2025-11-10 15:16:17,061 INFO: Waiting for execution to finish. Current state: RUNNING. Final status: UNDEFINED
2025-11-10 15:17:56,917 INFO: Waiting for execution to finish. Current state: AGGREGATING_LOGS. Final status: SUCCEEDED
2025-11-10 15:17:57,090 INFO: Waiting for log aggregation to finish.
2025-11-10 15:18:05,765 INFO: Execution finished successfully.


(Job('weather_1_offline_fg_materialization', 'SPARK'),
 {
   "success": true,
   "results": [
     {
       "success": true,
       "expectation_config": {
         "expectation_type": "expect_column_min_to_be_between",
         "kwargs": {
           "column": "wind_speed_10m_max",
           "min_value": -0.1,
           "max_value": 1000.0,
           "strict_min": true
         },
         "meta": {
           "expectationId": 739378
         }
       },
       "result": {
         "observed_value": 3.0758414268493652,
         "element_count": 28,
         "missing_count": null,
         "missing_percent": null
       },
       "meta": {
         "ingestionResult": "INGESTED",
         "validationTime": "2025-11-10T02:15:27.000377Z"
       },
       "exception_info": {
         "raised_exception": false,
         "exception_message": null,
         "exception_traceback": null
       }
     },
     {
       "success": true,
       "expectation_config": {
         "expectation_type"

In [16]:
df = weather_fg.read()
df.sort_values(by="date", ascending=False).head(40)

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (1.52s) 


,date,temperature_2m_mean,precipitation_sum,wind_speed_10m_max,wind_direction_10m_dominant,city
6578,2025-11-16 00:00:00+00:00,5.200000,0.0,12.313893,285.255157,skoghall
6565,2025-11-16 00:00:00+00:00,4.150000,0.0,12.144331,281.976105,årjäng
6580,2025-11-16 00:00:00+00:00,4.200000,0.0,12.594856,300.963684,jonsbyn
6583,2025-11-16 00:00:00+00:00,4.200000,0.0,12.727921,298.739685,nykroppa
6571,2025-11-15 00:00:00+00:00,2.750000,0.0,14.291592,229.085709,årjäng
6564,2025-11-15 00:00:00+00:00,2.600000,0.0,12.397806,205.820938,skoghall
6568,2025-11-15 00:00:00+00:00,1.400000,0.0,8.788720,214.992096,jonsbyn
6573,2025-11-15 00:00:00+00:00,1.150000,0.0,9.931042,226.468735,nykroppa
6584,2025-11-14 00:00:00+00:00,2.200000,0.0,19.228851,308.157257,nykroppa
6566,2025-11-14 00:00:00+00:00,3.350000,0.0,18.864824,311.905853,skoghall


In [17]:
df = air_quality_fg.read()
df.sort_values(by="date", ascending=False).head(40)

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (1.10s) 


,date,pm25,lagged_1,lagged_2,lagged_3,country,city,street,url
891,2025-11-10 00:00:00+00:00,19.00,12.13,5.95,6.00,sweden,skoghall,folkungavägen,https://api.waqi.info/feed/A401314
3286,2025-11-10 00:00:00+00:00,1.00,2.33,1.45,0.80,sweden,jonsbyn,acksjön,https://api.waqi.info/feed/A121810
1944,2025-11-10 00:00:00+00:00,7.00,7.79,6.67,5.90,sweden,årjäng,strand,https://api.waqi.info/feed/A87319
272,2025-11-10 00:00:00+00:00,2.00,3.78,3.53,2.42,sweden,nykroppa,timmersvägen,https://api.waqi.info/feed/A208483
3744,2025-11-09 00:00:00+00:00,2.33,1.45,0.80,1.50,sweden,jonsbyn,acksjön,https://api.waqi.info/feed/A121810
1428,2025-11-09 00:00:00+00:00,12.13,5.95,6.00,15.03,sweden,skoghall,folkungavägen,https://api.waqi.info/feed/A401314
1253,2025-11-09 00:00:00+00:00,3.78,3.53,2.42,8.07,sweden,nykroppa,timmersvägen,https://api.waqi.info/feed/A208483
3217,2025-11-09 00:00:00+00:00,7.79,6.67,5.90,4.00,sweden,årjäng,strand,https://api.waqi.info/feed/A87319
278,2025-11-08 00:00:00+00:00,1.45,0.80,1.50,2.08,sweden,jonsbyn,acksjön,https://api.waqi.info/feed/A121810
5148,2025-11-08 00:00:00+00:00,6.67,5.90,4.00,2.27,sweden,årjäng,strand,https://api.waqi.info/feed/A87319
